In [1]:
import os
os.chdir('..')

%load_ext autoreload
%autoreload 2

In [2]:
from functools import partial

import torch
from transformers import BertTokenizer, BertForNextSentencePrediction

from src.language_model_usage.winograd_schema_challenge import get_probability_of_next_sentence

I0426 16:05:46.701884 140090181846784 file_utils.py:35] PyTorch version 1.0.1.post2 available.
W0426 16:05:49.557264 140090181846784 __init__.py:28] To use data.metrics please install scikit-learn. See https://scikit-learn.org/stable/index.html


In [3]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertForNextSentencePrediction.from_pretrained('bert-base-uncased')

partial_get_probability_of_next_sentence = partial(get_probability_of_next_sentence, tokenizer, model)

I0426 16:05:51.388690 140090181846784 tokenization_utils.py:398] loading file https://s3.amazonaws.com/models.huggingface.co/bert/bert-base-uncased-vocab.txt from cache at /home/gabrielamelo/.cache/torch/transformers/26bc1ad6c0ac742e9b52263248f6d0f00068293b33709fae12320c0e35ccfbbb.542ce4285a40d23a559526243235df47c5f75c197f04f37d1a0c124c32c9a084
I0426 16:05:51.981636 140090181846784 file_utils.py:362] https://s3.amazonaws.com/models.huggingface.co/bert/bert-base-uncased-config.json not found in cache or force_download set to True, downloading to /tmp/tmpdn6ecrly
I0426 16:05:52.551770 140090181846784 file_utils.py:377] copying /tmp/tmpdn6ecrly to cache at /home/gabrielamelo/.cache/torch/transformers/4dad0251492946e18ac39290fcfe91b89d370fee250efe9521476438fe8ca185.7156163d5fdc189c3016baca0775ffce230789d7fa2a42ef516483e4ca884517
I0426 16:05:52.554802 140090181846784 file_utils.py:381] creating metadata file for /home/gabrielamelo/.cache/torch/transformers/4dad0251492946e18ac39290fcfe91b89d

In [15]:
text1 = "Where is the Eiffel Tower?"
text2 = "It is in Paris"
prediction = partial_get_probability_of_next_sentence(text1, text2)
print(prediction)

0.9999924898147583


In [4]:
text1 = "How old are you?"
text2 = "The Eiffel Tower is in Paris"
prediction = partial_get_probability_of_next_sentence(text1, text2)
print(prediction)

0.00041673300438560545


In [6]:
text1 = "How old are you?"
text2 = "I am 22 years old"
prediction = partial_get_probability_of_next_sentence(text1, text2)
print(prediction)

0.9999903440475464


In [13]:
text1_tokens = ['[CLS]'] + tokenizer.tokenize(text1) + ['[SEP]']
text2_tokens = tokenizer.tokenize(text2) + ['[SEP]']
text = text1_tokens + text2_tokens
indexed_tokens = tokenizer.convert_tokens_to_ids(text)
segments_ids = [0] * len(text1_tokens) + [1] * len(text2_tokens)

tokens_tensor = torch.tensor([indexed_tokens])
segments_tensors = torch.tensor([segments_ids])

model.eval()
prediction = model(tokens_tensor, token_type_ids=segments_tensors)
print(prediction)
prediction = prediction[0]  # tuple to tensor
print(prediction)
softmax = torch.nn.Softmax(dim=1)
prediction_sm = softmax(prediction)
print(prediction_sm)
print(prediction_sm[0][0].item())

(tensor([[ 5.9356, -5.6146]], grad_fn=<AddmmBackward>),)
tensor([[ 5.9356, -5.6146]], grad_fn=<AddmmBackward>)
tensor([[9.9999e-01, 9.6342e-06]], grad_fn=<SoftmaxBackward>)
0.9999903440475464


(tensor([[ 5.9356, -5.6146]], grad_fn=<AddmmBackward>),)

In [ ]:
tokenizer = BertTokenizer.from_pretrained('bert-base-multilingual-cased')
model = BertForNextSentencePrediction.from_pretrained('bert-base-multilingual-cased')

partial_get_probability_of_next_sentence = partial(get_probability_of_next_sentence, tokenizer, model)

In [ ]:
text1 = "Quantos anos você tem?"
text2 = "A Torre Eiffel fica em Paris"
prediction = partial_get_probability_of_next_sentence(text1, text2)
print(prediction)

In [ ]:
text1 = "Quantos anos você tem?"
text2 = "Eu tenho 22 anos"
prediction = partial_get_probability_of_next_sentence(text1, text2)
print(prediction)

In [ ]:
def get_sentence_breaks(first_sentence, second_sentence):
    for i in range(len(first_sentence.split())):
        if first_sentence.split()[i] != second_sentence.split()[i]:  # noqaE226
            break
    return i

In [ ]:
def test_get_sentence_breaks():
    first_sentence = 'The city councilmen refused the demonstrators a permit because the city councilmen feared violence.'
    second_sentence = 'The city councilmen refused the demonstrators a permit because the demonstrators feared violence.'
    i = get_sentence_breaks(first_sentence, second_sentence)
    assert ' '.join(first_sentence.split()[:i]) == \
        'The city councilmen refused the demonstrators a permit because the'
    assert ' '.join(second_sentence.split()[:i]) == \
        'The city councilmen refused the demonstrators a permit because the'
    assert ' '.join(first_sentence.split()[i:]) == \
        'city councilmen feared violence.'
    assert ' '.join(second_sentence.split()[i:]) == \
        'demonstrators feared violence.'
    
    first_sentence = 'Os vereadores recusaram a autorização aos manifestantes porque os vereadores temiam a violência.'
    second_sentence = 'Os vereadores recusaram a autorização aos manifestantes porque os manifestantes temiam a violência.'
    
    i = get_sentence_breaks(first_sentence, second_sentence)
    assert ' '.join(first_sentence.split()[:i]) == \
        'Os vereadores recusaram a autorização aos manifestantes porque os'
    assert ' '.join(second_sentence.split()[:i]) == \
        'Os vereadores recusaram a autorização aos manifestantes porque os'
    assert ' '.join(first_sentence.split()[i:]) == \
        'vereadores temiam a violência.'
    assert ' '.join(second_sentence.split()[i:]) == \
        'manifestantes temiam a violência.'
    
test_get_sentence_breaks()